In [40]:
#%% Import libraries
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import numpy as np

# Neural Network Components
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

warnings.filterwarnings('always')

#%% WANDB function creation
def wandb_block(params, sampling_interval, project, group):
    config_directory = params
    trial_number = config_directory["name"]
    run = wandb.init(
        settings=wandb.Settings(x_disable_stats=False, x_stats_sampling_interval = sampling_interval),
        # set the wandb project where this run will be logged
        name = f"Trial_{trial_number}",
        project = project,
        group = group,
        # track hyperparameters and run metadata with wandb.config
        config = config_directory
    )
    time.sleep(3.0)
    return run
      
#%% Neural Network Constructor
class NeuralNetworkConstructor:
    def __init__(self, inputs, filters=64, total_categories=2, conv_blocks=2, maxpooling_rate=5, bottleneck_length=2, 
                 downwards_activations=[None]*10, downwards_dropouts=[0.2]*5, downwards_regularizers=[None]*10,
                 bottleneck_activations=[None]*2, bottleneck_dropout=0.2, bottleneck_regularizers=[None]*2,
                 upwards_activations=[None]*10, upwards_dropouts=[0.2]*5, upwards_regularizers=[None]*10,
                 classifier_activation="relu"):
        self.inputs = inputs
        self.conv_blocks = conv_blocks
        self.maxpooling_rate = maxpooling_rate
        self.bottleneck_length = bottleneck_length
        self.filters =  filters
        self.complete_length = conv_blocks*maxpooling_rate
        self.residuals = [None]*conv_blocks
        self.total_categories = total_categories
        self.downwards_activations = downwards_activations
        self.downwards_dropouts = downwards_dropouts 
        self.downwards_regularizers = downwards_regularizers 
        
        self.bottleneck_activations = bottleneck_activations 
        self.bottleneck_dropout = bottleneck_dropout 
        self.bottleneck_regularizers = bottleneck_regularizers 
        
        self.upwards_activations = upwards_activations 
        self.upwards_dropouts = upwards_dropouts  
        self.upwards_regularizers = upwards_regularizers
        self.classifier_activation = classifier_activation
    
    def selected_regularizer(self, regularizer):
        if regularizer is None:
            selected_regularizer = "None"
        elif regularizer == "L1L2":
            selected_regularizer = f"keras.regularizers.{regularizer}(l1 = 1e-5, l2 = 1e-5)"
        else:
            selected_regularizer = f"keras.regularizers.{regularizer}(1e-5)"
        return selected_regularizer
        
    
    def downwards_path_iterator(self, block_inputs):
        x = block_inputs
        count=0
        for i in range(self.complete_length):
            current_filter = self.filters*(i//self.maxpooling_rate+1)
            current_activation = self.downwards_activations[i]
            current_regularizer = self.selected_regularizer(
                self.downwards_regularizers[i]
            )
            x = layers.Conv2D(filters=current_filter,
                              kernel_size=(3,3),
                              padding='valid',
                              activation=current_activation,
                              kernel_regularizer=eval(current_regularizer),
                              name=f'DownConv{i+1}')(x)
            x = layers.BatchNormalization(name=f'DownBN{i+1}')(x)
            
            if (i+1)%self.maxpooling_rate==0:
                self.residuals[count]=x
                x=layers.MaxPooling2D((2,2), padding='same',
                                      name=f'MaxPool{count+1}')(x)
                x=layers.Dropout(self.downwards_dropouts[count],
                                 name=f'DownDO{count+1}')(x)
                count+=1
                
        downwards_output = x
        return downwards_output
    
    def bottleneck_path_iterator(self, block_inputs):
        current_filter = self.filters*self.conv_blocks*2
        x = block_inputs
        for i in range(self.bottleneck_length):
            current_activation = self.bottleneck_activations[i]
            current_regularizer = self.selected_regularizer(
                self.bottleneck_regularizers[i]
            ) 
            x = layers.Conv2D(filters=current_filter,
                              kernel_size=(3,3),
                              padding='valid',
                              activation=current_activation,
                              kernel_regularizer=eval(current_regularizer),
                              name=f'BotConv{i+1}')(x)
            x = layers.BatchNormalization(name=f'BotBN{i+1}')(x)
        x = layers.Dropout(self.bottleneck_dropout,
                           name="BN_Dropout")(x)
        bottleneck_output = x
        return bottleneck_output

    def upwards_path_iterator(self, block_inputs):
        current_filter = self.filters*self.conv_blocks*2
        count = 0
        x = block_inputs
        for i in range(self.complete_length):
            current_activation = self.upwards_activations[i]
            current_regularizer = self.selected_regularizer(
                self.upwards_regularizers[i]
            )
            if i%self.maxpooling_rate==0:
                count+=1
                residual = self.residuals[self.conv_blocks-count]
                x = layers.Conv2DTranspose(filters=current_filter,
                                           kernel_size=(2,2),
                                           strides=(2,2),
                                           padding='same',
                                           name=f'UpSamp{i//self.maxpooling_rate+1}')(x)
                r_shape=residual.shape
                x_shape=x.shape
                #rescalamos el residual
                residual=layers.Resizing(x_shape[1], 
                                         x_shape[2],
                                         interpolation="bilinear")(residual)
                # unimos los residuals con las activaciones actuales
                x = layers.Concatenate(axis=-1)([x, residual])
                x=layers.Dropout(self.downwards_dropouts[count-1],
                                 name=f'UpDO{count}')(x)
                current_filter= current_filter//2
                
            x = layers.Conv2D(filters=current_filter,
                              kernel_size=(3,3),
                              padding='valid',
                              activation=current_activation,
                              kernel_regularizer=eval(current_regularizer),
                              name=f'UpConv{i+1}')(x)
            
            x = layers.BatchNormalization(name=f'UpBN{i+1}')(x)
        upwards_output = x
        return upwards_output

    def complete_outputs(self):
        x=self.inputs
        x=self.downwards_path_iterator(x)
        x=self.bottleneck_path_iterator(x)
        x=self.upwards_path_iterator(x)
        x=layers.Conv2D(filters=self.total_categories,
                        kernel_size=(1,1),
                        padding='valid',
                        activation=self.classifier_activation,
                        name='segmentation_map')(x)
        outputs=x
        return outputs

In [47]:
inputs=keras.Input((2048,1365,3))
conv_blocks=5
maxpooling_rate=3
a=conv_blocks*maxpooling_rate
params={'filters':64, 
        'total_categories':2, 
        'conv_blocks':conv_blocks, 
        'maxpooling_rate':maxpooling_rate, 
        'bottleneck_length':2,
        'downwards_activations': ['tanh']*a, 'downwards_dropouts': [0.2]*conv_blocks, 'downwards_regularizers': ['L1']*a,
        'bottleneck_activations': [None]*2, 'bottleneck_dropout': 0.2, 'bottleneck_regularizers':[None]*2,
        'upwards_activations':[None]*a, 'upwards_dropouts':[0.2]*conv_blocks, 'upwards_regularizers':[None]*a,
        'classifier_activation':'relu'}

creator = NeuralNetworkConstructor(inputs=inputs,**params)
outputs = creator.complete_outputs()
model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()


Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_38      │ (None, 2048,      │          0 │ -                 │
│ (InputLayer)        │ 1365, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownConv1 (Conv2D)  │ (None, 2046,      │      1,792 │ input_layer_38[0… │
│                     │ 1363, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownBN1             │ (None, 2046,      │        256 │ DownConv1[0][0]   │
│ (BatchNormalizatio… │ 1363, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownConv2 (Conv2D)  │ (None, 2044,      │     36,928 │ DownBN1[0][0]     │
│                     │ 1361, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownBN2             │ (None, 2044,      │        256 │ DownConv2[0][0]   │
│ (BatchNormalizatio… │ 1361, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownConv3 (Conv2D)  │ (None, 2042,      │     36,928 │ DownBN2[0][0]     │
│                     │ 1359, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownBN3             │ (None, 2042,      │        256 │ DownConv3[0][0]   │
│ (BatchNormalizatio… │ 1359, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MaxPool1            │ (None, 1021, 680, │          0 │ DownBN3[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownDO1 (Dropout)   │ (None, 1021, 680, │          0 │ MaxPool1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownConv4 (Conv2D)  │ (None, 1019, 678, │     73,856 │ DownDO1[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownBN4             │ (None, 1019, 678, │        512 │ DownConv4[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownConv5 (Conv2D)  │ (None, 1017, 676, │    147,584 │ DownBN4[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownBN5             │ (None, 1017, 676, │        512 │ DownConv5[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownConv6 (Conv2D)  │ (None, 1015, 674, │    147,584 │ DownBN5[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownBN6             │ (None, 1015, 674, │        512 │ DownConv6[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MaxPool2            │ (None, 508, 337,  │          0 │ DownBN6[0][0]     │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DownDO2 (Dropout)   │ (None, 508, 337,  │          0 │ MaxPool2[0][0]  

 Total params: 19,671,190 (75.04 MB)

 Trainable params: 19,659,150 (74.99 MB)

 Non-trainable params: 12,040 (47.03 KB)